<a href="https://colab.research.google.com/github/sejhig2/machineLearning_HONGONG_machine_propeller/blob/main/%EC%97%B0%EC%8A%B5_5_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

최적의 성능을 내는 하이퍼파라미터를 찾아보자

In [ ]:
# 검증세트 만들기 

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
train_sub_input, val_input, train_sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(train_sub_input.shape, val_input.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier( random_state= 42)
dt.fit(train_sub_input, train_sub_target)

print(dt.score(train_sub_input, train_sub_target))
print(dt.score(val_input, val_target))

In [ ]:
# 교차 검증 함수

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

In [ ]:
# splitter 분할기를 사용해서 데이터 섞기

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv= StratifiedKFold())
print(np.mean(scores['test_score']))

In [ ]:
splitter = StratifiedKFold( n_splits=10, shuffle = True)

scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

In [ ]:
from sklearn.model_selection import GridSearchCV
#min_impurity_decrease 매개변수의 범위 만들어주기
params  = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001] }

#교차검증할 객체 만들기
gs = GridSearchCV(DecisionTreeClassifier(max_depth=5),params,n_jobs = -1 )

In [ ]:
#훈련데이터로 그리드서치 훈련시키기.
gs.fit(train_input, train_target)

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

In [ ]:
print(gs.best_params_)

In [ ]:
best_index= np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001, 0.01),
          'max_depth': range(5,20,1),
          'min_samples_split':range(2,100,10)
          }

In [ ]:
gs= GridSearchCV(DecisionTreeClassifier(),params,n_jobs= -1)
gs.fit(train_input,train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(20)
np.unique(rgen.rvs(1000), return_counts = True)

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

In [ ]:
params ={'min_impurity_decrease':uniform(0.0001,0.001),
         'max_depth':randint(20,50),
         'min_samples_split': randint(2,25),
         'min_samples_leaf': randint(1,25),
         }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier() ,params, n_jobs= -1, n_iter =100)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))